In [28]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jul 19 19:55:34 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.52.01              Driver Version: 555.99         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti     On  |   00000000:01:00.0 Off |                  N/A |
|  0%   33C    P8             12W /  285W |    6934MiB /  12282MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [29]:
%%capture
!pip install datasets
!pip install --upgrade transformers
!pip install torchaudio
!pip install jiwer
!pip install accelerate -U

In [ ]:
%%capture
!apt install git-lfs

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset, load_metric, Audio
import os
name="nateraw/fsd50k"
train = load_dataset(name,  split="train", use_auth_token=True) 
train = train.remove_columns(["title", "mids", "description", "tags", "license", "uploader","split"])

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [5]:
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), 
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(train.remove_columns(["audio"]), num_examples=10)



,labels
0,"Gunshot_and_gunfire,Explosion"
1,Fire
2,"Piano,Keyboard_(musical),Musical_instrument,Music"
3,"Gasp,Breathing,Respiratory_sounds"
4,"Trumpet,Brass_instrument,Musical_instrument,Music"
5,"Guitar,Plucked_string_instrument,Musical_instrument,Music"
6,"Wind_instrument_and_woodwind_instrument,Musical_instrument,Music"
7,"Marimba_and_xylophone,Mallet_percussion,Percussion,Musical_instrument,Music"
8,"Wind_instrument_and_woodwind_instrument,Musical_instrument,Music"
9,"Wind_instrument_and_woodwind_instrument,Musical_instrument,Music"


In [6]:

import re
def clean_labels(batch):
    batch["labels"] = [re.sub(r'_', ' ', item) for item in batch["labels"]]
    batch["labels"] = [re.sub(r',', ' ', item) for item in batch["labels"]]
    batch["labels"] = [re.sub(r'\)', '', item) for item in batch["labels"]]
    batch["labels"] = [re.sub(r"\(", '', item) for item in batch["labels"]]

    return batch
train = train.map(clean_labels, batched=True, batch_size=1000) 

show_random_elements(train.remove_columns(["audio"]), num_examples=10)


,labels
0,Female speech and woman speaking Speech Human voice
1,Keyboard musical Musical instrument Music
2,Insect Wild animals Animal
3,Laughter Human voice
4,Liquid
5,Female speech and woman speaking Human voice Speech
6,Gasp Breathing Respiratory sounds
7,Burping and eructation
8,Dishes and pots and pans Chink and clink Domestic sounds and home sounds Glass
9,Human voice Cough Respiratory sounds


In [7]:
train = train.cast_column("audio", Audio(sampling_rate=16000))

In [8]:
import IPython.display as ipd
import numpy as np
import random
rand_int = random.randint(0, len(train)-1)

print(train[rand_int]["labels"])
ipd.Audio(data=train[rand_int]["audio"]["array"], autoplay=True, rate=16000)


In [9]:
rand_int = random.randint(0, len(train)-1)

print("Target text:", train[rand_int]["labels"])
print("Input array shape:", train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", train[rand_int]["audio"]["sampling_rate"])


In [10]:
from transformers import SeamlessM4TFeatureExtractor
feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0",stride = 2)


In [11]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [12]:
from diffusers import StableDiffusionPipeline
pipe=StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
text_encoder=pipe.components["text_encoder"]
tokenizer=pipe.components["tokenizer"]
print(tokenizer)
text_encoder.to(device)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 1024)
      (position_embedding): Embedding(77, 1024)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0): CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,), eps=1

In [13]:
def prepare_dataset(batch):
    audio = batch["audio"]
    labels= batch["labels"]
    features=feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"], truncation=True, padding='max_length', max_length=375)
    batch["input_features"],batch["attention_mask"] = features.input_features[0], features.attention_mask[0]
    batch["input_length"] = len(batch["input_features"])
    tokens=tokenizer(labels,return_tensors="pt", truncation=True, padding='max_length', max_length=48)
    with torch.no_grad():
        batch["label"] = text_encoder(**tokens.to(device)).last_hidden_state.squeeze(0).cpu()    
    return batch


In [14]:
train = train.map(prepare_dataset,remove_columns=train.column_names,batch_size=100)
text_encoder.to('cpu')


CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 1024)
      (position_embedding): Embedding(77, 1024)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0): CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,), eps=1

In [15]:
del pipe
del text_encoder
import gc
gc.collect()
torch.cuda.empty_cache()

In [16]:
train[0].keys()

dict_keys(['input_features', 'attention_mask', 'input_length', 'label'])

In [17]:
split_dataset = train.train_test_split(test_size=0.1)
train_subset = split_dataset['train']
test_subset = split_dataset['test']
print(f"Train subset size: {len(train_subset)}")
print(f"Test subset size: {len(test_subset)}")

In [18]:
import torch
from dataclasses import dataclass
@dataclass
class DataCollator:

    def __call__(self, features): 
        input_features = [torch.tensor(feature["input_features"], dtype=torch.float32) for feature in features]
        attention_mask = [torch.tensor(feature["attention_mask"], dtype=torch.float32) for feature in features]  
        label_features = [torch.tensor(feature["label"], dtype=torch.float32) for feature in features] 


        batch = {
            "input_features": torch.stack(input_features),
            "attention_mask": torch.stack(attention_mask), 
            "text_embeddings": torch.stack(label_features)[:, 1:, :]
        }
    

        return batch
data_collator = DataCollator()


In [19]:
from transformers import Wav2Vec2BertModel

model = Wav2Vec2BertModel.from_pretrained(
    "facebook/w2v-bert-2.0",
    add_adapter=True,
    adapter_kernel_size=3,
    adapter_stride=2,
    num_adapter_layers=2,
    layerdrop=0.0,
    )

print(next(model.parameters()).device)  


Some weights of Wav2Vec2BertModel were not initialized from the model checkpoint at facebook/w2v-bert-2.0 and are newly initialized: ['adapter.layers.0.ffn.intermediate_dense.bias', 'adapter.layers.0.ffn.intermediate_dense.weight', 'adapter.layers.0.ffn.output_dense.bias', 'adapter.layers.0.ffn.output_dense.weight', 'adapter.layers.0.ffn_layer_norm.bias', 'adapter.layers.0.ffn_layer_norm.weight', 'adapter.layers.0.residual_conv.bias', 'adapter.layers.0.residual_conv.weight', 'adapter.layers.0.residual_layer_norm.bias', 'adapter.layers.0.residual_layer_norm.weight', 'adapter.layers.0.self_attn.linear_k.bias', 'adapter.layers.0.self_attn.linear_k.weight', 'adapter.layers.0.self_attn.linear_out.bias', 'adapter.layers.0.self_attn.linear_out.weight', 'adapter.layers.0.self_attn.linear_q.bias', 'adapter.layers.0.self_attn.linear_q.weight', 'adapter.layers.0.self_attn.linear_v.bias', 'adapter.layers.0.self_attn.linear_v.weight', 'adapter.layers.0.self_attn_conv.bias', 'adapter.layers.0.self_a

In [20]:
import torch
import torch.nn as nn
def compute_metrics(pred):
    predictions = pred.predictions
    labels = pred.label_ids

    tensor_predictions = torch.from_numpy(predictions)
    tensor_labels = torch.from_numpy(labels)

    cosine_similarities = torch.cosine_similarity(tensor_predictions, tensor_labels, dim=-1)
    avg_cos_sim = cosine_similarities.mean()

    mse_loss = nn.MSELoss()
    E_mse = mse_loss(tensor_predictions, tensor_labels)    

    euclidean_distances = torch.norm(tensor_predictions - tensor_labels, dim=-1)
    avg_euclidean_dist = euclidean_distances.mean().item()
    
    return {
        'cosine': avg_cos_sim,
        'E_mse': E_mse,
        'euclidean_distance': avg_euclidean_dist,
    }

In [21]:
from transformers import Trainer
import torch.nn.functional as F

def cosine_similarity_loss(embeddings1, embeddings2, margin=0.5):
    cos_sim = torch.cosine_similarity(embeddings1, embeddings2,dim=-1)
    loss = torch.mean(1 - cos_sim + margin)
    return loss   


def Contrastive_loss(embeddings1, embeddings2, temperature=0.15):
 
    cos_sim = torch.cosine_similarity(embeddings1.unsqueeze(1), embeddings2.unsqueeze(0), dim=-1)
    cos_sim = cos_sim / temperature 
    labels = torch.arange(embeddings1.size(0)).unsqueeze(1).repeat(1, embeddings1.size(1)).to(embeddings1.device)    
    loss = F.cross_entropy(cos_sim, labels)   
    return loss

class TrainBert(Trainer):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
  
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("text_embeddings")
        outputs = model(**inputs)
        outputs=outputs.last_hidden_state
        loss = Contrastive_loss(outputs, labels)
        outputs = (loss, outputs)
        return (loss, outputs) if return_outputs else loss
    
    

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="train/w2v-bert_sd_conditioning",
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  gradient_accumulation_steps=16,
  evaluation_strategy="steps",
  num_train_epochs=8,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=288,
  eval_steps=144,  
  logging_steps=144,
  learning_rate=2e-5,
  warmup_steps=200,
  save_total_limit=4,
  push_to_hub=False,
  label_names =["text_embeddings"],
  logging_dir='train/logs/run1_full_training_full_data_(contrastive_loss_T_015_adapter_3*3)',
  dataloader_num_workers=10, 
  eval_accumulation_steps=10,
  #load_best_model_at_end=True,
)

/home/youssef/miniconda3/envs/ldmmm/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
trainer = TrainBert(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=test_subset,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)



In [24]:
torch.cuda.empty_cache() 
import gc
gc.collect()
trainer.train()
#trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss,Cosine,E Mse,Euclidean Distance
144,1.913700,1.734753,0.085479,1.756623,42.263260
288,1.680900,1.613458,0.116268,1.659532,41.095066
432,1.551000,1.506113,0.147984,1.553063,39.733353
576,1.475700,1.440376,0.156046,1.438201,38.234638
720,1.405200,1.405672,0.172788,1.314867,36.545670
864,1.378100,1.372734,0.181113,1.272299,35.930172
1008,1.325900,1.350391,0.190992,1.223559,35.238632
1152,1.307500,1.338883,0.198897,1.205876,34.976425
1296,1.267900,1.323583,0.204958,1.178422,34.572132
1440,1.259100,1.314878,0.202416,1.158960,34.271767


TrainOutput(global_step=2304, training_loss=1.3572947515381708, metrics={'train_runtime': 19282.704, 'train_samples_per_second': 15.296, 'train_steps_per_second': 0.119, 'total_flos': 3.356981348459176e+19, 'train_loss': 1.3572947515381708, 'epoch': 7.998264265567368})